### Задание 2

Соберите единый отчет по работе всего приложения. В отчете должна быть информация и по ленте новостей, и по сервису отправки сообщений.   

Продумайте, какие метрики необходимо отобразить в этом отчете? Как можно показать их динамику?  Приложите к отчету графики или файлы, чтобы сделать его более наглядным и информативным.    

Сохраните скрипт сборки отчета в свой репозиторий в GitLab для автоматизации отчетности. Автоматизируйте отправку отчета с помощью GitLab CI/CD. Отчет должен приходить ежедневно в 11:00 в чат. 


In [2]:
import telegram
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import io
import logging
import pandas as pd
import pandahouse
from read_db import Getch
import os

In [3]:
sns.set()

In [1]:
%env REPORT_CHAT_ID=1
%env REPORT_BOT_TOKEN=4

env: REPORT_CHAT_ID=1
env: REPORT_BOT_TOKEN=4


In [12]:
bot = telegram.Bot(token=os.environ.get("REPORT_BOT_TOKEN"))

In [13]:
chat_id = int(os.environ.get("REPORT_CHAT_ID"))

### Метрики
За вчера:
- Количество пользователей, использующих ленту
- Количество пользователей, использующих сообщения
- Количество пользователей, использующих только ленту
- Количество пользователей, использующих только сообщения

Графики:
- Активные пользователи в ленте
- Активные пользователи в сообщениях
- Доля пользователей сообщений от пользователей ленты

### Данные

In [ ]:
feed_data_query = """
SELECT
    distinct user_id,
    toDate(time) date
FROM simulator_20220420.feed_actions
WHERE dateDiff('day', toDate(time), toDate(today())) <= 7
    AND toDate(time) <> toDate(today())
"""

feed_data = Getch(feed_data_query).df

In [16]:
feed_data.head()

,user_id,date
0,143899,2022-04-30
1,118355,2022-04-30
2,143644,2022-04-30
3,147312,2022-04-30
4,150888,2022-04-30


In [17]:
message_data_query = """
SELECT
    distinct user_id,
    toDate(time) date
FROM simulator_20220420.message_actions
WHERE dateDiff('day', toDate(time), toDate(today())) <= 7
    AND toDate(time) <> toDate(today())
"""

message_data = Getch(message_data_query).df

In [19]:
message_data.head()

,user_id,date
0,8038,2022-04-26
1,11591,2022-04-26
2,116065,2022-04-26
3,10361,2022-04-26
4,11736,2022-04-26


### Графики

In [31]:
feed_agg_data = feed_data.groupby("date").agg({"user_id": "count"}).reset_index()
feed_agg_data

,date,user_id
0,2022-04-26,14073
1,2022-04-27,18248
2,2022-04-28,18666
3,2022-04-29,18608
4,2022-04-30,19129
5,2022-05-01,18613
6,2022-05-02,18572


In [28]:
msg_agg_data = message_data.groupby("date").agg({"user_id": "count"}).reset_index()
msg_agg_data

,date,user_id
0,2022-04-26,1566
1,2022-04-27,1498
2,2022-04-28,2242
3,2022-04-29,1575
4,2022-04-30,1569
5,2022-05-01,2373
6,2022-05-02,1681


In [60]:
feed_and_msg_agg_data = feed_agg_data.merge(msg_agg_data, on='date', suffixes=['_feed', '_msg'])
feed_and_msg_agg_data

,date,user_id_feed,user_id_msg
0,2022-04-26,14073,1566
1,2022-04-27,18248,1498
2,2022-04-28,18666,2242
3,2022-04-29,18608,1575
4,2022-04-30,19129,1569
5,2022-05-01,18613,2373
6,2022-05-02,18572,1681


In [62]:
feed_and_msg_agg_data['part_of_msg_users'] = feed_and_msg_agg_data['user_id_msg'] / feed_and_msg_agg_data['user_id_feed']

In [74]:
report_date_begin = (pd.Timestamp('now') - pd.DateOffset(days=7)).date()
report_date_end = (pd.Timestamp('now') - pd.DateOffset(days=1)).date()

In [26]:
annotation_xytext = (0, 0) # distance from text to points (x, y)
annotatin_ha = 'center' # horizontal alignment can be left, right or center
annotation_font_size = 9
annotation_bbox = dict(boxstyle="round", fc="w", alpha=0.5)

In [66]:
fig, axes = plt.subplots(3, 1, figsize=(8, 11), dpi=200)
fig.suptitle(f"Report on {report_date_begin} – {report_date_end}", fontsize=15)
fig.subplots_adjust(hspace=0.3)

############### Активные пользователи в ленте ###############
sns.lineplot(
    ax=axes[0],
    data=feed_agg_data, 
    x='date', 
    y='user_id', 
    )

axes[0].set(xlabel=None)
axes[0].set(ylabel='Количество пользователей')


for x, y in zip(feed_agg_data["date"], feed_agg_data["user_id"]):
    label = f"{y:,}"
    axes[0].annotate(
        label, # this is the text
        (x, y), # these are the coordinates to position the label
        size=annotation_font_size,
        textcoords="offset points", # how to position the text
        xytext=annotation_xytext, # distance from text to points (x, y)
        ha=annotatin_ha, # horizontal alignment can be left, right or center
        bbox=annotation_bbox,
    ) 

axes[0].set_title("Активные пользователи в мессенджере")
axes[0].set(ylim=(None, feed_agg_data['user_id'].max() * 1.03))


############### Активные пользователи в сообщениях ###############
sns.lineplot(
    ax=axes[1],
    data=msg_agg_data, 
    x='date', 
    y='user_id', 
    )

axes[1].set(xlabel=None)
axes[1].set(ylabel='Количество пользователей')


for x, y in zip(msg_agg_data["date"], msg_agg_data["user_id"]):
    label = f"{y:,}"
    axes[1].annotate(
        label, # this is the text
        (x, y), # these are the coordinates to position the label
        size=annotation_font_size,
        textcoords="offset points", # how to position the text
        xytext=annotation_xytext, # distance from text to points (x, y)
        ha=annotatin_ha, # horizontal alignment can be left, right or center
        bbox=annotation_bbox,
    ) 

axes[1].set_title("Активные пользователи в ленте")
axes[1].set(ylim=(None, msg_agg_data['user_id'].max() * 1.03))


############### Доля пользователей сообщений от пользователей ленты ###############
sns.lineplot(
    ax=axes[2],
    data=feed_and_msg_agg_data, 
    x='date', 
    y='part_of_msg_users', 
    )

axes[2].set(xlabel=None)
axes[2].set(ylabel='Доля пользователей')


for x, y in zip(feed_and_msg_agg_data["date"], feed_and_msg_agg_data["part_of_msg_users"]):
    label = f"{y:.2}"
    axes[2].annotate(
        label, # this is the text
        (x, y), # these are the coordinates to position the label
        size=annotation_font_size,
        textcoords="offset points", # how to position the text
        xytext=annotation_xytext, # distance from text to points (x, y)
        ha=annotatin_ha, # horizontal alignment can be left, right or center
        bbox=annotation_bbox,
    ) 

axes[2].set_title("Активные пользователи в ленте")
axes[2].set(ylim=(None, feed_and_msg_agg_data['part_of_msg_users'].max() * 1.03))


# Sending picture
plot_object = io.BytesIO()
plt.savefig(plot_object)
plot_object.name = 'test_plot.png'
plot_object.seek(0)
plt.close()

bot.send_photo(chat_id=chat_id, photo=plot_object)

### Данные за последний день

In [67]:
last_date = feed_data['date'].max()

feed_last_day = feed_data[feed_data['date'] == last_date]

In [71]:
feed_last_day

,user_id,date
101704,14224,2022-05-02
101705,54120,2022-05-02
101706,58034,2022-05-02
101707,58118,2022-05-02
101708,141283,2022-05-02
...,...,...
124966,58923,2022-05-02
124967,110063,2022-05-02
124968,131904,2022-05-02
124969,157390,2022-05-02


In [72]:
msg_last_day = message_data[message_data['date'] == last_date]

In [73]:
msg_last_day

,user_id,date
10823,8741,2022-05-02
10824,11236,2022-05-02
10825,11822,2022-05-02
10826,116358,2022-05-02
10827,8256,2022-05-02
...,...,...
12499,122459,2022-05-02
12500,2601,2022-05-02
12501,11968,2022-05-02
12502,122575,2022-05-02


In [85]:
text_message = f"""Отчет по Ленте и Сообщениям за {last_date.date()}\n
`Пользователи ленты             {feed_last_day['user_id'].nunique():,}
Пользователи сообщений         {msg_last_day['user_id'].nunique():,}
Использовали только ленту      {feed_last_day[~feed_last_day['user_id'].isin(msg_last_day['user_id'])]['user_id'].nunique():,}
Использовали только сообщения  {msg_last_day[~msg_last_day['user_id'].isin(feed_last_day['user_id'])]['user_id'].nunique():,}`
"""

In [86]:
text_message

'Отчет по Ленте и Сообщениям за 2022-05-02\n\n`Пользователи ленты             18,572\nПользователи сообщений         1,681\nИспользовали только ленту      18,398\nИспользовали только сообщения  1,507`\n'

In [82]:
bot.send_message(chat_id=chat_id, text=text_message, parse_mode='markdown')